# Import Libraries 

In [1]:
!pip install transformers

In [2]:
!pip install transformers datasets tqdm


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 6.7 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.5.1
    Uninstalling fsspec-2025.5.1:
      Successfully uninstalled fsspec-2025.5.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.8.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
cesium 0.12.4 requires numpy<3.0,>=2.0, but you have numpy 1.26.4 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-cupti-cu12 12.5.82 which is incompatible.
torch 2.6.0+cu124 requires nv

In [3]:
import numpy as np
import pandas as pd 
import seaborn as sns 
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import matplotlib.pyplot as plt
import torch
from transformers import BertForQuestionAnswering ,pipeline

2025-08-02 09:27:48.975822: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1754126869.200112      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1754126869.267792      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


# Load SQuAD v1.1 dataset from Hugging Face Datasets

In [4]:
from datasets import load_dataset
# Load SQuAD v1.1
squad_dataset = load_dataset("squad")


README.md: 0.00B [00:00, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/14.5M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/1.82M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/87599 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/10570 [00:00<?, ? examples/s]

#  Load Pretrained BERT QA model

In [5]:
from transformers import BertTokenizer

In [6]:
model=BertForQuestionAnswering.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')

config.json:   0%|          | 0.00/443 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.34G [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-large-uncased-whole-word-masking-finetuned-squad were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [7]:
tokenizer=BertTokenizer.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [8]:
# Create pipeline for easy inference
qa_pipeline = pipeline("question-answering", model=model, tokenizer=tokenizer)

Device set to use cpu


# Run the model on a few validation examples

In [9]:
for example in squad_dataset["validation"].select(range(5)):
    question = example["question"]
    context = example["context"]
    answer_text = example["answers"]["text"][0]

    result = qa_pipeline(question=question, context=context)

    print(f"\nQ: {question}")
    print(f"Predicted Answer: {result['answer']}")
    print(f"True Answer: {answer_text}")
    print("-" * 60)



Q: Which NFL team represented the AFC at Super Bowl 50?
Predicted Answer: Denver Broncos
True Answer: Denver Broncos
------------------------------------------------------------

Q: Which NFL team represented the NFC at Super Bowl 50?
Predicted Answer: Carolina Panthers
True Answer: Carolina Panthers
------------------------------------------------------------

Q: Where did Super Bowl 50 take place?
Predicted Answer: Levi's Stadium in the San Francisco Bay Area at Santa Clara, California.
True Answer: Santa Clara, California
------------------------------------------------------------

Q: Which NFL team won Super Bowl 50?
Predicted Answer: Denver Broncos
True Answer: Denver Broncos
------------------------------------------------------------

Q: What color was used to emphasize the 50th anniversary of the Super Bowl?
Predicted Answer: gold-themed
True Answer: gold
------------------------------------------------------------


# Evaluate on the full validation set (optional but advanced)

In [ ]:
!pip install evaluate

import evaluate

# Load squad metric
metric = evaluate.load("squad")

# Run on 100 examples only to keep it fast (you can remove the select to run full)
for example in squad_dataset["validation"].select(range(100)):
    question = example["question"]
    context = example["context"]
    true_answer = example["answers"]["text"][0]

    result = qa_pipeline(question=question, context=context)
    predicted_answer = result["answer"]

    metric.add(
        prediction={"id": example["id"], "prediction_text": predicted_answer},
        reference={"id": example["id"], "answers": example["answers"]}
    )

# Get final results
final_score = metric.compute()
print("\nEvaluation Results:")
print(f"Exact Match (EM): {final_score['exact_match']:.2f}")
print(f"F1 Score: {final_score['f1']:.2f}")



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.8 MB/s eta 0:00:00
